<a href="https://colab.research.google.com/github/dbanerjee181/SAT-Solver/blob/main/dpll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# my code starts from here
# Reading a file in the DIMACS format and
# create the clause from it
# The Dimacs format goes as follows
# c means comment
# p cnf indicates the number of variable
# followed by number of clauses
# function signature is
# [clauses,clause_count,variable_count] = read_dimacs_file(filename)

import re


def read_dimacs_file(filename):
    """Reads a DIMACS file and returns the clauses and variable count."""
    clauses = []
    variable_count = 0
    clause_count = 0
    with open(filename, 'r') as file:
      for line in file:
        line = line.strip()
        if line.startswith(' ') or line.startswith('\t') or (not line):
          continue
        if line.startswith('0'):
          continue
        if line.startswith('c') or line.startswith('%'):
          continue
        if(re.match(r'p cnf',line)):
          variable_count = int(line.split()[2])
          clause_count = int(line.split()[3])
        else:
          line.strip()
          clause_list = [int(x) for x in line.split()[:-1]]
          clauses.append(clause_list)
    return clauses,variable_count,clause_count


In [2]:
# Given the clauses and assignment this function checks if
# it there is any unit clause and then updates the assignment
# vector
# function signature
# check_unit_clause(clauses,assignment)
# This updates the assignment
# dictionary in place


def check_unit_clause(clauses, assignment):
  """Checks for unit clauses and updates the assignment vector."""
  for clause in clauses:
    if len(clause) == 1:
      variable = abs(clause[0])
      if clause[0] > 0:
        assignment[variable] = True
        clauses.remove(clause)
      else:
        assignment[variable] = False
        clauses.remove(clause)
  return clauses, assignment



In [3]:
#Simplify the clause based on assignment
#Checking procedure example

def simplify(clauses, assignment):
  new_clauses = []
  for clause in clauses:
    new_clause = []
    satisfied = False
    for literal in clause:
      variable = abs(literal)
      if variable in assignment:
        if (literal > 0 and assignment[variable]) or \
           (literal < 0 and not assignment[variable]):
          satisfied = True
          break  # Clause is satisfied
        elif (literal > 0 and not assignment[variable]) or \
             (literal < 0 and assignment[variable]):
          continue #literal is false

      else:
        new_clause.append(literal)
        satisfied = False

    if not satisfied:
        new_clauses.append(new_clause)

  return new_clauses

clauses = [[1, 2], [-1, 3], [-2, -3]]
assignment = {1: True}
new_clauses = simplify(clauses, assignment)
print(new_clauses)

[[3], [-2, -3]]


In [4]:
def find_pure_literal(clauses, assignment):
    literals = {}
    mono_literals = {}
    for clause in clauses:
        for literal in clause:
          variable = abs(literal)
          if variable not in assignment:
            if literal not in literals:
              literals[literal] = 0
            literals[literal]+=1
    for literal in literals:
      if -literal not in literals:
        mono_literals[literal] = literals[literal]

    return mono_literals

clauses = [[1, 2], [-1], [-3], [4], [4,5], [7,-1]]
assignment = {1: True, 2: True}
find_pure_literal(clauses, assignment)

{-3: 1, 4: 2, 5: 1, 7: 1}

In [5]:
def unassigned_literal(clauses):
    literals = {}
    for clause in clauses:
        for literal in clause:
          variable = abs(literal)
          if variable not in literals:
            literals[variable] = [0,0]
          if literal > 0 :
            literals[variable] = [literals[variable][0]+1,literals[variable][1]]
          elif literal < 0:
            literals[variable] = [literals[variable][0],literals[variable][1]+1]

    sorted_literals = dict(sorted(literals.items(), key=lambda item: sum(item[1]), reverse=True))
    return sorted_literals



clauses = [[1, 2], [3,-2], [1,-3],[-2],[-2],[-2]]
assignment = {1: True}
stats = unassigned_literal(clauses)
# Accessing the first element
first_literal = next(iter(stats))
print("First literal:", first_literal)

First literal: 2


In [42]:
import time

initial_assignment = {}
initial_clauses = []

def dpll(clauses, assignment={}, iterations=0,initial_assignment={},initial_clauses=[]):
  print(clauses)
  print(assignment)
  print(iterations)



  if(iterations == 0):
    if(assignment):
      initial_assignment = assignment.copy()
      initial_clauses = clauses.copy()

  print(f"Initial assignments : {initial_assignment}")
  print(f"Initial clauses : {initial_clauses}")

  pure_literals = find_pure_literal(clauses, assignment)
  #pure literal assignments

  for literal in pure_literals:
    variable = abs(literal)
    if literal > 0:
      assignment[variable] = True
    else:
      assignment[variable] = False

  new_clauses = simplify(clauses, assignment)
  # Check if all clauses are satisfied
  if not new_clauses:
    return assignment, iterations
  # Check if any clause is empty (unsatisfiable)

  if(any(not clause for clause in new_clauses)and bool(initial_assignment)):
    print("here")
    assignment = {}
    new_clauses = initial_clauses.copy()
    initial_assignment = {}
    initial_clauses = []

  if (any(not clause for clause in new_clauses)and bool(not(initial_assignment))):
    print("meeting this condition")
    return None, iterations

  stats = unassigned_literal(new_clauses)
  if not stats:
    return assignment, iterations
  priority_var = next(iter(stats))
  new_assignment = assignment.copy()
  if (stats[priority_var][0] >=  stats[priority_var][1]):
    new_assignment[priority_var] = True
  elif (stats[priority_var][0] < stats[priority_var][1]):
    new_assignment[priority_var] = False

  new_clauses_true = simplify(new_clauses, new_assignment)
  updated_clauses,unit_assignment = check_unit_clause(new_clauses_true, new_assignment)
  result_true = dpll(updated_clauses, unit_assignment, iterations + 1,initial_assignment,initial_clauses)
  if result_true:
    return result_true

  new_assignment[priority_var] = not new_assignment[priority_var]
  new_clauses_false = simplify(new_clauses, new_assignment)
  updated_clauses, unit_assignment = check_unit_clause(new_clauses_false, new_assignment)
  result_false = dpll(updated_clauses, unit_assignment, iterations + 1,initial_assignment,initial_clauses)
  if result_false:
    return result_false


  return None

start_time = time.time()  # Record the start time
#clauses,variable_count,clause_count  = read_dimacs_file("satTestFile.cnf")
#print(clauses)
clauses = [[-1, -2, -3], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-2, -3, -4], [-1, -2, -3],[-1,-2,3]]
assignment = {}
iterations = 0
result, iterations= dpll(clauses, assignment, iterations)

if not result:
  print("RESULT: UNSAT")
else:
  print("RESULT: SAT")
  print(sorted(result.items()))
  print("Time taken:", time.time() - start_time)
  print("Calls of DPLL:", iterations)

[[-1, -2, -3], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-1, -3, -4], [-2, -3, -4], [-1, -2, -3], [-1, -2, 3]]
{}
0
Initial assignments : {}
Initial clauses : []
RESULT: SAT
[(1, False), (2, False), (4, False)]
Time taken: 0.0009412765502929688
Calls of DPLL: 0


In [ ]:
# implement Stalmarck reduction in the
# preprocessing of CNF formulae

[2, 1, 4, -1, 3]


In [ ]:
def main():
  [clauses,variable_count,clause_count] = read_dimacs_file("uf20-0999.cnf")
  print(f"The number of variables are {variable_count},The number of clauses are {clause_count}")
  print(clauses)
  assignment = stalmarck(clauses)
  print(assignment)
  clauses = [[1, 2],[2],[-1, 3]]
  assignment = {1: True, 3: True}
  check_unit_clause(clauses, assignment)
  print(assignment)


In [ ]:
if __name__ == "__main__":
  main()

The number of variables are 20,The number of clauses are 91
[[13, 15, -5], [5, -13, -9], [-2, -13, -9], [-16, 18, 19], [-6, 14, 5], [-7, 4, 11], [-15, 19, 14], [20, -3, -19], [-20, -9, -11], [2, -6, -10], [13, -6, 3], [9, 11, -8], [-9, -19, 7], [-17, -20, 12], [-17, 4, -16], [20, -5, -7], [-10, -4, 11], [5, 9, -1], [17, -1, 19], [-1, -2, -6], [15, 17, -19], [15, -14, 18], [-16, -15, 19], [-16, 6, -15], [-20, 5, -3], [-10, 20, 16], [-6, 17, -7], [7, 2, -16], [-18, 5, 13], [-17, 13, 12], [-14, -6, -12], [14, -2, -9], [3, -14, -17], [-1, 18, -6], [14, -18, -8], [7, -3, -19], [-18, -20, -5], [20, 12, 15], [5, 3, 15], [16, -6, -18], [8, 5, -18], [4, 6, -15], [6, 3, 4], [9, -11, -12], [12, 9, 5], [4, 18, -8], [16, -8, 1], [3, 1, -7], [15, -9, -4], [-5, -3, -10], [-16, -12, -19], [12, -3, -16], [4, -18, -6], [5, -7, -3], [15, -1, -5], [-16, 9, 10], [-9, 17, 5], [-2, 4, 10], [16, 9, -11], [1, -7, -15], [-20, -8, 3], [3, 9, 17], [-11, 9, 6], [8, 16, 19], [2, 8, -3], [-5, 15, 18], [1, 16, 2], [-